## Harmony Py Library
### Job Pause/Resume Example

In [ ]:
import sys
sys.path.append('..')

# Install harmony-py requirements.  Not necessary if you ran `pip install harmony-py` in your kernel  
!{sys.executable} -m pip install -q -r ../requirements/core.txt

from harmony import BBox, Client, Collection, Request, Environment

In [ ]:
harmony_client = Client(env=Environment.UAT)  # assumes .netrc usage

collection = Collection(id='C1234088182-EEDTEST')
request = Request(
    collection=collection,
    max_results=101
)

In [ ]:
# submit an async request for processing and return the job_id
# big requests get automatically paused after generating a preview of the results
job_id = harmony_client.submit(request)

In [ ]:
# checking the status of the job we see that it is 'previewing'
harmony_client.status(job_id)

In [ ]:
# 'wait_for_processing()' will wait while the job is in the 'previewing' state then
# warns that the job is paused before exiting
harmony_client.wait_for_processing(job_id, show_progress=True)

In [ ]:
# checking the status we see that the job is paused
harmony_client.status(job_id)

In [ ]:
# 'result_json()' will not wait for paused jobs and just returns any available results.
results = harmony_client.download_all(job_id, directory='/tmp', overwrite=True)
count = 0
for r in results:
    count += 1
print(f'Got {count} results')

In [ ]:
# we can resume the job with 'resume()'
harmony_client.resume(job_id)

In [ ]:
# checking the status we see that the job is running again
harmony_client.status(job_id)

In [ ]:
# we can pause the job with 'pause()'.
harmony_client.pause(job_id)

In [ ]:
# checking the status we see that the job is paused again
harmony_client.status(job_id)

In [ ]:
# We can resume the job again
harmony_client.resume(job_id)

In [ ]:
# 'wait_for_processing()' will show resumed progress
harmony_client.wait_for_processing(job_id, show_progress=True)

In [ ]:
# 'download_all()' now has access to the full results
results = harmony_client.download_all(job_id, directory='/tmp', overwrite=True)
count = 0
for r in results:
    count += 1
print(f'Got {count} results')

In [ ]:
# Attempting to pause a completed job will result in an error
harmony_client.pause(job_id)

In [ ]:
# Attempting to resume a completed job will also result in an error
harmony_client.resume(job_id)

In [ ]:
# we can use the 'skip_preview' parameter to tell Harmony to skip the auto-pause/preview and just start running
harmony_client = Client(env=Environment.UAT)  # assumes .netrc usage

collection = Collection(id='C1234088182-EEDTEST')
request = Request(
    collection=collection,
    max_results=101,
    skip_preview=True
)

In [ ]:
# submit an async request for processing and return the job_id
# big requests get automatically paused after generating a preview of the results
job_id = harmony_client.submit(request)

In [ ]:
# checking the status we see that the job is running
harmony_client.status(job_id)

In [ ]:
# we can now use'wait_for_processing()' to wait until the job completes
harmony_client.wait_for_processing(job_id, show_progress=True)